<a href="https://colab.research.google.com/github/Keerthana0309/Movie-Lens-Recommendation-System/blob/master/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd

In [27]:
data = pd.read_csv('ratings.csv')
data.head(10)

,userId,movieId,rating,timestamp
0,1,2,3.5,1.112486e+09
1,1,29,3.5,1.112485e+09
2,1,32,3.5,1.112485e+09
3,1,47,3.5,1.112485e+09
4,1,50,3.5,1.112485e+09
5,1,112,3.5,1.094786e+09
6,1,151,4.0,1.094786e+09
7,1,223,4.0,1.112486e+09
8,1,253,4.0,1.112485e+09
9,1,260,4.0,1.112485e+09


In [28]:
movie_title_genre = pd.read_csv('movies.csv')
movie_title_genre.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [29]:
data = data.merge(movie_title_genre,on='movieId',how='left')
data.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1.112486e+09,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,1.112485e+09,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,1.112485e+09,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,1.112485e+09,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,1.112485e+09,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,1,112,3.5,1.094786e+09,Rumble in the Bronx (Hont faan kui) (1995),Action|Adventure|Comedy|Crime
6,1,151,4.0,1.094786e+09,Rob Roy (1995),Action|Drama|Romance|War
7,1,223,4.0,1.112486e+09,Clerks (1994),Comedy
8,1,253,4.0,1.112485e+09,Interview with the Vampire: The Vampire Chroni...,Drama|Horror
9,1,260,4.0,1.112485e+09,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


**Feature Engineering**

In [34]:
Average_ratings = pd.DataFrame(data.groupby('title')['rating'].mean())
Average_ratings.head(10)

,rating
title,
"""Great Performances"" Cats (1998)",2.875000
$5 a Day (2008),3.300000
'71 (2014),3.250000
'Hellboy': The Seeds of Creation (2004),3.125000
'Neath the Arizona Skies (1934),3.500000
'R Xmas (2001),3.000000
'Round Midnight (1986),3.785714
'Salem's Lot (2004),2.714286
'Til There Was You (1997),2.881579


In [35]:
Average_ratings['Total Ratings'] = pd.DataFrame(data.groupby('title')['rating'].count())
Average_ratings.head(10)

,rating,Total Ratings
title,,
"""Great Performances"" Cats (1998)",2.875000,8
$5 a Day (2008),3.300000,5
'71 (2014),3.250000,2
'Hellboy': The Seeds of Creation (2004),3.125000,4
'Neath the Arizona Skies (1934),3.500000,2
'R Xmas (2001),3.000000,1
'Round Midnight (1986),3.785714,7
'Salem's Lot (2004),2.714286,7
'Til There Was You (1997),2.881579,38


**Building the Recommender**
*Calculating the Correlation*

In [37]:
movie_user = data.pivot_table(index='userId',columns='title',values='rating')
movie_user.head(10)

title,"""Great Performances"" Cats (1998)",$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),(Untitled) (2009),*batteries not included (1987),+1 (2013),"...All the Marbles (California Dolls, The) (1981)",...And God Spoke (1993),...And Justice for All (1979),.45 (2006),1-900 (06) (1994),10 (1979),10 Items or Less (2006),10 Rillington Place (1971),10 Things I Hate About You (1999),10 Years (2011),10 to Midnight (1983),"10,000 BC (2008)",100 Feet (2008),100 Girls (2000),100 Rifles (1969),"1000 Eyes of Dr. Mabuse, The (Die 1000 Augen des Dr. Mabuse) (1960)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11-11-11 (11-11-11: The Prophecy) (2011),...,Zero Dark Thirty (2012),Zero Day (2002),Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Ziggy Stardust and the Spiders from Mars (1973),Zindagi Na Milegi Dobara (2011),Zizek! (2005),Zodiac (2007),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980),Zombie Island Massacre (1984),"Zombie Lake (Lac des morts vivants, Le) (Zombies Lake) (Lake of the Living Dead, The) (1981)",Zombie Strippers! (2008),Zombie and the Ghost Train (Zombie ja Kummitusjuna) (1991),Zombieland (2009),Zona Zamfirova (2002),Zookeeper (2011),Zoolander (2001),Zoom (2006),Zoot Suit (1981),Zorba the Greek (Alexis Zorbas) (1964),Zorro (1975),"Zorro, the Gay Blade (1981)",Zozo (2005),Zulu (1964),Zulu Dawn (1979),Zus & Zo (2001),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nos amours (1983),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [38]:
correlations = movie_user.corrwith(movie_user['Toy Story (1995)'])
correlations.head(10)

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
"Great Performances" Cats (1998)                NaN
$5 a Day (2008)                                 NaN
'71 (2014)                                 1.000000
'Hellboy': The Seeds of Creation (2004)         NaN
'Neath the Arizona Skies (1934)                 NaN
'R Xmas (2001)                                  NaN
'Round Midnight (1986)                    -1.000000
'Salem's Lot (2004)                       -1.000000
'Til There Was You (1997)                  0.258998
'burbs, The (1989)                         0.053051
dtype: float64

*Now remove all the empty values and merge the total ratings to the correlation table.*

In [39]:
recommendation = pd.DataFrame(correlations,columns=['Correlation'])
recommendation.dropna(inplace=True)
recommendation = recommendation.join(Average_ratings['Total Ratings'])
recommendation.head()

,Correlation,Total Ratings
title,,
'71 (2014),1.000000,2
'Round Midnight (1986),-1.000000,7
'Salem's Lot (2004),-1.000000,7
'Til There Was You (1997),0.258998,38
"'burbs, The (1989)",0.053051,165


**Testing The Recommendation System**

In [40]:
recc = recommendation[recommendation['Total Ratings']>100].sort_values('Correlation',ascending=False).reset_index()

In [41]:
recc = recc.merge(movie_title_genre, on='title', how='left')
recc.head(10)

,title,Correlation,Total Ratings,movieId,genres
0,Toy Story (1995),1.000000,2634,1,Adventure|Animation|Children|Comedy|Fantasy
1,Toy Story 2 (1999),0.769238,1214,3114,Adventure|Animation|Children|Comedy|Fantasy
2,Finding Nemo (2003),0.559134,1257,6377,Adventure|Animation|Children|Comedy
3,"Monsters, Inc. (2001)",0.540128,1282,4886,Adventure|Animation|Children|Comedy|Fantasy
4,"Bug's Life, A (1998)",0.535217,1108,2355,Adventure|Animation|Children|Comedy
5,Toy Story 3 (2010),0.526737,313,78499,Adventure|Animation|Children|Comedy|Fantasy|IMAX
6,Cars (2006),0.483913,246,45517,Animation|Children|Comedy
7,All Quiet on the Western Front (1930),0.470331,127,1927,Action|Drama|War
8,Stuart Little (1999),0.457717,273,3157,Children|Comedy|Fantasy
9,"Incredibles, The (2004)",0.457288,1087,8961,Action|Adventure|Animation|Children|Comedy
